 ## Глава 4: Форматирование набора данных

 ### Содержание главы

 В этой главе мы:

 *   Поймем важность определения корректного шаблона чата.
 *   Обсудим несколько альтернативных подходов к форматированию, включая пользовательские функции форматирования и шаблоны.
 *   Настроим токенизатор и слой эмбеддингов модели.
 *   Изучим упакованные наборы данных и различные сборщики данных (collators) для их загрузки.

 ### Настройка окружения

In [1]:
# Если вы используете Colab
!pip install datasets bitsandbytes trl


In [2]:
# Если вы используете Jupyter Template от runpod.io
#!pip install datasets bitsandbytes trl transformers peft huggingface-hub accelerate safetensors pandas matplotlib


 ### Импорт библиотек

In [3]:
import torch
from peft import prepare_model_for_kbit_training, get_peft_model, LoraConfig
from datasets import load_dataset, Dataset
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig, DataCollatorForLanguageModeling, DataCollatorWithPadding, DataCollatorWithFlattening, BitsAndBytesConfig
from trl import setup_chat_format #, DataCollatorForCompletionOnlyLM - удален в версии 0.20
from trl.data_utils import pack_dataset
from trl.extras.dataset_formatting import FORMAT_MAPPING, instructions_formatting_function, conversations_formatting_function
# from trl.trainer import ConstantLengthDataset - удален в версии 0.20


In [4]:
# Если вы используете Colab, необходимо загрузить замену удаленных функций
!wget https://raw.githubusercontent.com/dvgodoy/FineTuningLLMs/refs/heads/main/compatibility_functions.py

from compatibility_functions import DataCollatorForCompletionOnlyLM, ConstantLengthDataset


--2025-12-11 16:22:00--  https://raw.githubusercontent.com/dvgodoy/FineTuningLLMs/refs/heads/main/compatibility_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15950 (16K) [text/plain]
Saving to: ‘compatibility_functions.py.5’

compatibility_funct 100%[===================>]  15.58K  --.-KB/s    in 0.008s  

2025-12-11 16:22:01 (1.93 MB/s) - ‘compatibility_functions.py.5’ saved [15950/15950]



 ### Цель форматирования

 Мы форматируем набор данных, чтобы предоставить структуру и контекстные указания LLM. Мы можем легко направлять ее поведение (например, при инструктивной настройке), аккуратно оборачивая каждый компонент — промпт пользователя и ответ модели — соответствующими тегами и специальными токенами.

 ### Суть форматирования

 ![](https://github.com/dvgodoy/FineTuningLLMs/blob/main/images/ch4/base_prompt.png?raw=True)

 <center>Рисунок 4.1 - Предсказание следующего токена базовой моделью</center>



 ![](https://github.com/dvgodoy/FineTuningLLMs/blob/main/images/ch4/fine_tuned_prompt.png?raw=True)

 <center>Рисунок 4.2 - Настроенная модель, активируемая шаблоном ответа</center>



 ![](https://github.com/dvgodoy/FineTuningLLMs/blob/main/images/ch4/chat_prompt_new.png?raw=True)

 <center>Рисунок 4.3 - Чат-модель, использующая шаблон чата</center>



 ![](https://github.com/dvgodoy/FineTuningLLMs/blob/main/images/ch4/chat_example_new.png?raw=True)

 <center>Рисунок 4.4 - Общая структура шаблона чата</center>

 ### Итог предыдущих глав

In [5]:
supported = torch.cuda.is_bf16_supported(including_emulation=False)
compute_dtype = (torch.bfloat16 if supported else torch.float32)

nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=compute_dtype
)

from huggingface_hub import snapshot_download

# Download entire repository
snapshot_download(
    repo_id="facebook/opt-350m",
    local_dir="./opt-350m-model",
    ignore_patterns=["*.h5", "*.ot", "*.msgpack"],  # Skip unnecessary formats
    local_dir_use_symlinks=False
)

# Load from local directory
model_dir = "./opt-350m-model"

model_q4 = AutoModelForCausalLM.from_pretrained(model_dir,
                                                device_map='cuda:0',
                                                torch_dtype=compute_dtype,
                                                quantization_config=nf4_config)

model_q4 = prepare_model_for_kbit_training(model_q4)

config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)
peft_model = get_peft_model(model_q4, config)


/home/loschilov_aa/.conda/envs/pytorchbook/lib/python3.10/site-packages/huggingface_hub/file_download.py:982: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!


 ### Применение шаблонов

 ****
 **Итог раздела "Применение шаблонов"**



 У вас есть три варианта форматирования набора данных:

 1. Ваш набор данных представлен в **одном из двух форматов, поддерживаемых классом `STTrainer`** (диалоговый или инструктивный):
    *   Ваш **токенизатор должен иметь настроенный шаблон чата**.
    *   Не нужно определять функцию форматирования или форматировать набор данных перед обучением.
    *   **ВАЖНО**: **инструктивный формат больше не поддерживается должным образом в последних версиях пакета `trl`**.
 2. Вы хотите использовать **пользовательскую функцию форматирования** (см. "BYOFF, Bring Your Own Formatting Function"):
    *   Пользовательская функция должна быть предоставлена как **аргумент `formatting_func` класса `SFTTrainer`** (см. Главу 5).
    *   Ваша функция форматирования **должна обрабатывать батчи данных**.
      *   Протестируйте ее, вызвав метод `map()` набора данных с `batched=True`.
     *   Не нужно применять функцию к набору данных перед обучением.
     *   Если ваш токенизатор уже **имеет шаблон чата**:
       *   Вы можете вызвать его метод `apply_chat_template()` в своей функции.
       *   Придерживайтесь общего формата шаблона (шаблоны инструкции и ответа).
       *   Если шаблон не включает его, **вы можете добавить токен `EOS` в конец отформатированного вывода**.
    *   Если ваш токенизатор **не имеет шаблона чата**:
      *   Вы свободны в определении общего формата, включая шаблоны инструкции и ответа (см. "Продвинутый уровень — BYOT, Bring Your Own Template").

 3. Ваш набор данных **уже отформатирован** (см. "BYOFD, Bring Your Own Formatted Data"):
    *   Столбец, содержащий отформатированные данные, должен быть предоставлен как **аргумент `dataset_text_field` класса `SFTTrainer`** (см. Главу 5).
    *   Даже если вы можете использовать свою функцию форматирования для предобработки набора данных, она не будет использоваться классом тренера.
    *   Убедитесь, что ваши **данные совместимы с шаблоном токенизатора**.
 ****

In [6]:
from huggingface_hub import snapshot_download

# Загружаем все файлы в локальную директорию
snapshot_download(
    repo_id="microsoft/Phi-3-mini-4k-instruct",
    local_dir="./phi3-mini-tokenizer",
    allow_patterns=[
        "tokenizer.json",
        "tokenizer_config.json",
        "special_tokens_map.json",
        "vocab.json",
        "merges.txt",
        "added_tokens.json"
    ]
)

# Затем загружаем из локальной директории
from transformers import AutoTokenizer
tokenizer_phi = AutoTokenizer.from_pretrained("./phi3-mini-tokenizer")
print(tokenizer_phi.chat_template)


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

{% for message in messages %}{% if message['role'] == 'system' %}{{'<|system|>
' + message['content'] + '<|end|>
'}}{% elif message['role'] == 'user' %}{{'<|user|>
' + message['content'] + '<|end|>
'}}{% elif message['role'] == 'assistant' %}{{'<|assistant|>
' + message['content'] + '<|end|>
'}}{% endif %}{% endfor %}{% if add_generation_prompt %}{{ '<|assistant|>
' }}{% else %}{{ eos_token }}{% endif %}


In [7]:
messages = [
    {'role': 'system', 'content': 'You are a helpful AI assistant.'},
    {'role': 'user', 'content': 'What is the capital of Argentina?'},
    {'role': 'assistant', 'content': 'Buenos Aires.'}
]

formatted = tokenizer_phi.apply_chat_template(conversation=messages,
                                          tokenize=False,
                                          add_generation_prompt=False)
print(formatted)


<|system|>
You are a helpful AI assistant.<|end|>
<|user|>
What is the capital of Argentina?<|end|>
<|assistant|>
Buenos Aires.<|end|>
<|endoftext|>


In [8]:
inference_input = tokenizer_phi.apply_chat_template(conversation=messages[:-1],
                                          tokenize=False,
                                          add_generation_prompt=True)
print(inference_input)


<|system|>
You are a helpful AI assistant.<|end|>
<|user|>
What is the capital of Argentina?<|end|>
<|assistant|>



 #### Поддерживаемые форматы

 ##### Диалоговый формат

In [9]:
conversation_ds = Dataset.from_list([{'messages': messages}])
conversation_ds.features


{'messages': List({'content': Value('string'), 'role': Value('string')})}

In [10]:
FORMAT_MAPPING['chatml'] == conversation_ds.features['messages']


True

In [11]:
formatting_func = conversations_formatting_function(tokenizer_phi, messages_field='messages')

print(formatting_func(conversation_ds[0]))


<|system|>
You are a helpful AI assistant.<|end|>
<|user|>
What is the capital of Argentina?<|end|>
<|assistant|>
Buenos Aires.<|end|>
<|endoftext|>


 ```python
 # функция форматирования для диалогового формата
 def format_dataset(examples):
     if isinstance(examples[messages_field][0], list):
         output_texts = []
         for i in range(len(examples[messages_field])):
             output_texts.append(tokenizer.apply_chat_template(examples[messages_field][i], tokenize=False))

         return output_texts
     else:
         return tokenizer.apply_chat_template(examples[messages_field], tokenize=False)
 ```

 ##### Инструктивный формат

 ****

 **ВАЖНОЕ ОБНОВЛЕНИЕ**: к сожалению, в более новых версиях библиотеки `trl` инструктивный формат больше не поддерживается должным образом, что приводит к тому, что шаблон чата не применяется к набору данных. Чтобы избежать этой проблемы, рекомендуется использовать диалоговый формат.

 ****



 ```python
 instructions = [{'prompt': 'What is the capital of Argentina?',
                  'completion': 'Buenos Aires.'}]

 instruction_ds = Dataset.from_list(instructions)
 instruction_ds.features

 {'prompt': Value('string'), 'completion': Value('string')}

 FORMAT_MAPPING['instruction'] == instruction_ds.features

 True

 formatting_func = instructions_formatting_function(tokenizer_phi)
 formatting_func


 trl.extras.dataset_formatting.instructions_formatting_function.<locals>.format_dataset
 def format_dataset(examples)
 ```

In [12]:
# Адаптировано из trl.extras.dataset_formatting.instructions_formatting_function
# Преобразует набор данных из формата промпт/завершение (больше не поддерживается)
# в диалоговый формат
def format_dataset(examples):
    if isinstance(examples["prompt"], list):
        output_texts = []
        for i in range(len(examples["prompt"])):
            converted_sample = [
                {"role": "user", "content": examples["prompt"][i]},
                {"role": "assistant", "content": examples["completion"][i]},
            ]
            output_texts.append(converted_sample)
        return {'messages': output_texts}
    else:
        converted_sample = [
            {"role": "user", "content": examples["prompt"]},
            {"role": "assistant", "content": examples["completion"]},
        ]
        return {'messages': converted_sample}


In [13]:
batch_prompts_completions = {
    'prompt': ['What is the capital of Argentina?',
               'What is the capital of the United States?'],
    'completion': ['Buenos Aires.',
                    'Washington D.C.']
}


In [14]:
batch_messages = format_dataset(batch_prompts_completions)['messages']
batch_messages


[[{'role': 'user', 'content': 'What is the capital of Argentina?'},
  {'role': 'assistant', 'content': 'Buenos Aires.'}],
 [{'role': 'user', 'content': 'What is the capital of the United States?'},
  {'role': 'assistant', 'content': 'Washington D.C.'}]]

 #### BYOFF (Принеси свою собственную функцию форматирования)

In [15]:
def byo_formatting_func1(examples):
    messages = examples["messages"]
    output_texts = tokenizer_phi.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
    return output_texts


In [16]:
ds_msg = Dataset.from_dict({'messages': batch_messages})
ds_msg.map(lambda v: tokenizer_phi(byo_formatting_func1(v)), batched=True)


Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Dataset({
    features: ['messages', 'input_ids', 'attention_mask'],
    num_rows: 2
})

In [17]:
def byo_formatting_func2(examples):
    response_template = '### Answer:'
    text = f"### Question: {examples['prompt']}\n{response_template} {examples['completion']}"
    text += tokenizer_phi.eos_token
    return text


In [18]:
ds_prompt = Dataset.from_dict(batch_prompts_completions)
print(byo_formatting_func2(ds_prompt[0]))


### Question: What is the capital of Argentina?
### Answer: Buenos Aires.<|endoftext|>


In [19]:
# это вызовет исключение
# ds_prompt.map(lambda v: tokenizer_phi(byo_formatting_func2(v)), batched=True)


In [20]:
def byo_formatting_func3(examples):
    output_texts = []
    response_template = '### Answer:'
    for i in range(len(examples['prompt'])):
        text = f"### Question: {examples['prompt'][i]}\n {response_template} {examples['completion'][i]}"
        text += tokenizer_phi.eos_token
        output_texts.append(text)
    return output_texts


In [21]:
ds_prompt.map(lambda v: tokenizer_phi(byo_formatting_func3(v)), batched=True)


Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Dataset({
    features: ['prompt', 'completion', 'input_ids', 'attention_mask'],
    num_rows: 2
})

 #### BYOFD (Принеси свои собственные отформатированные данные)

In [22]:
def byofd_formatting_func(examples):
    messages = examples["messages"]
    output_texts = tokenizer_phi.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
    return {'text': output_texts}


In [23]:
formatted_ds = ds_msg.map(byofd_formatting_func, batched=True)
formatted_ds['text']


Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Column(['<|user|>\nWhat is the capital of Argentina?<|end|>\n<|assistant|>\nBuenos Aires.<|end|>\n<|endoftext|>', '<|user|>\nWhat is the capital of the United States?<|end|>\n<|assistant|>\nWashington D.C.<|end|>\n<|endoftext|>'])

 #### Сводка выбора формата

 ![](https://github.com/dvgodoy/FineTuningLLMs/blob/main/images/ch4/formatting_flow.png?raw=True)



 <center>Рисунок 4.5 - Выбор правильной конфигурации для ваших потребностей в форматировании</center>

 ### Токенизатор

 ****

 **Итог раздела "Токенизатор"**

 *   **Словарь токенизатора** обычно **короче, чем слой эмбеддингов модели**.
     *   Разница в размере состоит, буквально, из «пустых слотов», которые можно использовать для **создания новых токенов без изменения размера** слоя эмбеддингов.
     *   **Размер слоя эмбеддингов** часто является **кратным степени двойки** (32, 64 и т.д.) для оптимизации **выделения памяти**.
 *   Токен `EOS` должен **использоваться исключительно для обозначения конца текста** и ничего более.
     *   Использование токена `EOS` для дополнения (padding) может привести к _бесконечной генерации токенов_.
 *   Токен `PAD` часто не определен, но он может понадобиться:
     *   **НЕ** назначайте токен `EOS` в качестве токена `PAD`.
     *   Если токен `UNK` определен, можно назначить его в качестве токена `PAD`.
     *   Если токен `UNK` не определен, создайте новый специальный токен в качестве токена `PAD`.
     *   **ВНИМАНИЕ**: Если токен `PAD` остался **неопределенным**, многие библиотеки **по умолчанию назначат ему токен `EOS`**!
 *   Для **генеративных** моделей **дополнение** должно выполняться с **левой** стороны.
     *   Дополнение _справа_ научит модель генерировать _бесконечные последовательности токенов дополнения_.
     *   Во многих руководствах используется `tokenizer.padding_side='right'` из-за сообщений о проблемах с переполнением в классе `SFTTrainer`.
       *   Это допустимо **только если вы используете упаковку или сборщики, подобные упаковке** (см. раздел "Упакованный набор данных") вместо стандартного дополнения.

 *   Если вы **создаете новые специальные токены**, теоретически, вам также следует **настроить слой эмбеддингов** (поскольку вы используете эти «пустые слоты»).
     *   На практике ваша модель _может_ все еще работать, если вы **оставите эмбеддинги замороженными**.
     *   Даже если представление новых токенов _случайно_ (их эмбеддинги не обучены), другие обучаемые части модели могут научиться использовать их «как есть».

 ****

In [24]:
tokenizer_phi = AutoTokenizer.from_pretrained("./phi3-mini-tokenizer")
config_phi = AutoConfig.from_pretrained("./phi3-mini-tokenizer", trust_remote_code=True)


In [25]:
tokenizer_phi("Let's tokenize this sentence!")


{'input_ids': [2803, 29915, 29879, 5993, 675, 445, 10541, 29991], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

 #### Словарь

In [26]:
len(tokenizer_phi), config_phi.vocab_size


(32011, 32064)

In [27]:
sorted(tokenizer_phi.vocab.items(), key=lambda t: -t[1])[:11]


[('<|user|>', 32010),
 ('<|placeholder6|>', 32009),
 ('<|placeholder5|>', 32008),
 ('<|end|>', 32007),
 ('<|system|>', 32006),
 ('<|placeholder4|>', 32005),
 ('<|placeholder3|>', 32004),
 ('<|placeholder2|>', 32003),
 ('<|placeholder1|>', 32002),
 ('<|assistant|>', 32001),
 ('<|endoftext|>', 32000)]

In [28]:
tokenizer_phi.eos_token, tokenizer_phi.eos_token_id


('<|endoftext|>', 32000)

 #### Специальные токены

In [29]:
tokenizer_phi.all_special_tokens


['<s>', '<|endoftext|>', '<unk>']

In [30]:
tokenizer_phi.special_tokens_map


{'bos_token': '<s>',
 'eos_token': '<|endoftext|>',
 'unk_token': '<unk>',
 'pad_token': '<|endoftext|>'}

In [31]:
tokenizer_phi.cls_token, tokenizer_phi.sep_token, tokenizer_phi.mask_token


(None, None, None)

In [32]:
tokenizer_phi.add_special_tokens({'cls_token': '<cls>', 'sep_token': '<sep>', 'mask_token': '<mask>'})
tokenizer_phi.special_tokens_map


{'bos_token': '<s>',
 'eos_token': '<|endoftext|>',
 'unk_token': '<unk>',
 'sep_token': '<sep>',
 'pad_token': '<|endoftext|>',
 'cls_token': '<cls>',
 'mask_token': '<mask>'}

In [33]:
sorted(tokenizer_phi.vocab.items(), key=lambda t: -t[1])[:13]


[('<mask>', 32013),
 ('<sep>', 32012),
 ('<cls>', 32011),
 ('<|user|>', 32010),
 ('<|placeholder6|>', 32009),
 ('<|placeholder5|>', 32008),
 ('<|end|>', 32007),
 ('<|system|>', 32006),
 ('<|placeholder4|>', 32005),
 ('<|placeholder3|>', 32004),
 ('<|placeholder2|>', 32003),
 ('<|placeholder1|>', 32002),
 ('<|assistant|>', 32001)]

 #### Токен `EOS`

In [34]:
tokenizer_phi.pad_token = tokenizer_phi.unk_token
tokenizer_phi.pad_token_id = tokenizer_phi.unk_token_id

tokenizer_phi.special_tokens_map


{'bos_token': '<s>',
 'eos_token': '<|endoftext|>',
 'unk_token': '<unk>',
 'sep_token': '<sep>',
 'pad_token': '<unk>',
 'cls_token': '<cls>',
 'mask_token': '<mask>'}

 ```python

 # Обновление конфигурации модели для измененного токена PAD

 if getattr(model, "config", None) is not None:

     model.config.pad_token_id = tokenizer_phi.pad_token_id

 if (getattr(model, "generation_config", None) s not None):

     model.config.pad_token_id = tokenizer_phi.pad_token_id

 ```

 #### Токен `PAD`

In [35]:
tokenizer_phi.pad_token, tokenizer_phi.padding_side


('<unk>', 'left')

 ### Сборщики данных (Data Collators)

 ****

 **Итог раздела "Сборщики данных"**

 *   Вы можете указать аргумент `data_collator` в классе `SFTTrainer` (см. Главу 5).

 *   `DataCollatorForLanguageModeling` — это **стандартный** сборщик для класса `SFTTrainer`:

     *   Он автоматически **копирует идентификаторы токенов в качестве меток**.

     *   Он **не смещает метки**, поскольку это **обрабатывается автоматически моделью**.

     *   Он включает полный текст (и промпт, и завершение) в качестве меток, что идеально для инструктивной настройки.

 *   Если вы выполняете дальнейшую тонкую настройку инструктивной или чат-модели, вы можете использовать `DataCollatorForCompletionOnlyLM` для **обучения только на ответе модели (завершении)**.

     *   Он также копирует идентификаторы токенов в качестве меток, но **маскирует токены промпта, заменяя их идентификаторы на `-100`**.

     *   В **одном взаимодействии** (один промпт и одно завершение) **шаблона ответа достаточно** для определения завершения.

     *   При **нескольких взаимодействиях** (последовательность промптов и завершений) необходимы как **шаблон инструкции, так и шаблон ответа**, чтобы правильно идентифицировать и замаскировать токены промпта.

 ****

In [36]:
dataset = load_dataset("dvgodoy/yoda_sentences", split="train")
dataset = dataset.rename_column("sentence", "prompt")
dataset = dataset.rename_column("translation_extra", "completion")
# преобразует пары промпт/завершение в диалоговые сообщения
dataset = dataset.map(format_dataset)
dataset = dataset.remove_columns(["prompt", "completion", "translation"])
len(dataset), dataset[0]


(720,
 {'messages': [{'content': 'The birch canoe slid on the smooth planks.',
    'role': 'user'},
   {'content': 'On the smooth planks, the birch canoe slid. Yes, hrrrm.',
    'role': 'assistant'}]})

In [37]:
# formatting_func = instructions_formatting_function(tokenizer_phi)
formatting_func = conversations_formatting_function(tokenizer_phi, messages_field='messages')
dataset = dataset.map(lambda row: {'text': formatting_func(row)}, batched=True, batch_size=32)
sequences = dataset['text']
print(sequences[:2])


['<|user|>\nThe birch canoe slid on the smooth planks.<|end|>\n<|assistant|>\nOn the smooth planks, the birch canoe slid. Yes, hrrrm.<|end|>\n<|endoftext|>', '<|user|>\nGlue the sheet to the dark blue background.<|end|>\n<|assistant|>\nGlue the sheet to the dark blue background, you must.<|end|>\n<|endoftext|>']


In [38]:
tokenized_dataset = dataset.map(lambda row: tokenizer_phi(row['text']))
tokenized_dataset = tokenized_dataset.select_columns(['input_ids'])


 #### `DataCollatorWithPadding`

In [39]:
pad_collator = DataCollatorWithPadding(tokenizer_phi)
pad_dloader = DataLoader(tokenized_dataset, batch_size=2, collate_fn=pad_collator)
pad_batch = next(iter(pad_dloader))
pad_batch


{'input_ids': tensor([[32010,   450, 29773,   305,   508,  7297,  2243,   333,   373,   278,
         10597,   715,  1331, 29889, 32007, 32001,  1551,   278, 10597,   715,
          1331, 29892,   278, 29773,   305,   508,  7297,  2243,   333, 29889,
          3869, 29892,   298, 21478,  1758, 29889, 32007, 32000],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
         32010,  8467,   434,   278,  9869,   304,   278,  6501,  7254,  3239,
         29889, 32007, 32001,  8467,   434,   278,  9869,   304,   278,  6501,
          7254,  3239, 29892,   366,  1818, 29889, 32007, 32000]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

 #### А где же мои метки?

 ![](https://github.com/dvgodoy/FineTuningLLMs/blob/main/images/ch4/shift_labels.png?raw=True)



 <center>Рисунок 4.6 - Входные данные и соответствующие им смещенные метки</center>

 #### `DataCollatorForLanguageModeling`

In [40]:
lm_collator = DataCollatorForLanguageModeling(tokenizer_phi, mlm=False)
lm_dloader = DataLoader(tokenized_dataset, batch_size=2, collate_fn=lm_collator)
lm_batch = next(iter(lm_dloader))
lm_batch


{'input_ids': tensor([[32010,   450, 29773,   305,   508,  7297,  2243,   333,   373,   278,
         10597,   715,  1331, 29889, 32007, 32001,  1551,   278, 10597,   715,
          1331, 29892,   278, 29773,   305,   508,  7297,  2243,   333, 29889,
          3869, 29892,   298, 21478,  1758, 29889, 32007, 32000],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
         32010,  8467,   434,   278,  9869,   304,   278,  6501,  7254,  3239,
         29889, 32007, 32001,  8467,   434,   278,  9869,   304,   278,  6501,
          7254,  3239, 29892,   366,  1818, 29889, 32007, 32000]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'labels': tensor([[32010,   450, 29773,   305,   508,  7297,  2243,   333,   373,   278,
   

 #### `DataCollatorForCompletionOnlyLM`

 ****

 **ВАЖНОЕ ОБНОВЛЕНИЕ**: `DataCollatorForCompletionOnlyLM` был удален в `trl` версии 0.20. Этот сборщик маскировал промпт пользователя, чтобы обучать только на завершении модели, исключая токены промпта из вычисления потерь, используя шаблон ответа для обнаружения начала завершения.



 В новых версиях эта логика встроена: токены автоматически игнорируются при вычислении потерь, если в объекте `SFTConfig` установлены `completion_only_loss` или `assistant_only_loss`. Хотя это проще, данный подход менее гибок, поскольку зависит от совместимого шаблона чата.



 Чтобы сохранить гибкость и продемонстрировать процесс маскирования, который теперь обрабатывается внутренне, я скопировал исходную реализацию `DataCollatorForCompletionOnlyLM` в `compatibility_functions.py` (импортирован в начале этой главы) и буду продолжать использовать ее для обучения только на завершении.

 ****

In [41]:
response_template = '<|assistant|>' # id токена 32001
completion_collator = DataCollatorForCompletionOnlyLM(response_template=response_template,
                                                      tokenizer=tokenizer_phi)
completion_dloader = DataLoader(tokenized_dataset, batch_size=2, collate_fn=completion_collator)
completion_batch = next(iter(completion_dloader))
completion_batch


{'input_ids': tensor([[32010,   450, 29773,   305,   508,  7297,  2243,   333,   373,   278,
         10597,   715,  1331, 29889, 32007, 32001,  1551,   278, 10597,   715,
          1331, 29892,   278, 29773,   305,   508,  7297,  2243,   333, 29889,
          3869, 29892,   298, 21478,  1758, 29889, 32007, 32000],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
         32010,  8467,   434,   278,  9869,   304,   278,  6501,  7254,  3239,
         29889, 32007, 32001,  8467,   434,   278,  9869,   304,   278,  6501,
          7254,  3239, 29892,   366,  1818, 29889, 32007, 32000]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'labels': tensor([[ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
   

In [42]:
labels = completion_batch['labels'][0]
valid_tokens = (labels >= 0)
tokenizer_phi.decode(labels[valid_tokens])



'On the smooth planks, the birch canoe slid. Yes, hrrrm.<|end|><|endoftext|>'

 ##### Несколько взаимодействий

In [43]:
dummy_chat = """<|user|>Hello
<|assistant|>How are you?
<|user|>I'm fine! You?
<|assistant|>I'm fine too!
<|endoftext|>"""

dummy_ds = Dataset.from_dict({'text': [dummy_chat]})
dummy_ds = dummy_ds.map(lambda row: tokenizer_phi(row['text'])).select_columns(['input_ids'])


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [44]:
completion_dloader = DataLoader(dummy_ds, batch_size=1, collate_fn=completion_collator)
completion_batch = next(iter(completion_dloader))
completion_batch


{'input_ids': tensor([[32010, 15043,    13, 32001,  1128,   526,   366, 29973,    13, 32010,
           306, 29915, 29885,  2691, 29991,   887, 29973,    13, 32001,   306,
         29915, 29885,  2691,  2086, 29991,    13, 32000]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1]]), 'labels': tensor([[ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,   306,
         29915, 29885,  2691,  2086, 29991,    13, 32000]])}

In [45]:
labels = completion_batch['labels']
tokenizer_phi.decode(labels[labels >= 0])


"I'm fine too!\n<|endoftext|>"

In [46]:
instruction_template = '<|user|>'
response_template = '<|assistant|>'
completion_collator = DataCollatorForCompletionOnlyLM(instruction_template=instruction_template,
                                                      response_template=response_template,
                                                      tokenizer=tokenizer_phi)
completion_dloader = DataLoader(dummy_ds, batch_size=1, collate_fn=completion_collator)
completion_batch = next(iter(completion_dloader))
completion_batch


{'input_ids': tensor([[32010, 15043,    13, 32001,  1128,   526,   366, 29973,    13, 32010,
           306, 29915, 29885,  2691, 29991,   887, 29973,    13, 32001,   306,
         29915, 29885,  2691,  2086, 29991,    13, 32000]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1]]), 'labels': tensor([[ -100,  -100,  -100,  -100,  1128,   526,   366, 29973,    13,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,   306,
         29915, 29885,  2691,  2086, 29991,    13, 32000]])}

In [47]:
labels = completion_batch['labels']
tokenizer_phi.decode(labels[labels >= 0])


"How are you?\n I'm fine too!\n<|endoftext|>"

 #### Смещение меток

 ```python
 if labels is not None:
     # перемещаем метки на правильное устройство для параллелизма модели
     labels = labels.to(lm_logits.device)
     # мы выполняем предсказание следующего токена; смещаем предсказанные оценки и входные идентификаторы на один
     shift_logits = lm_logits[:, :-1, :].contiguous()
     labels = labels[:, 1:].contiguous()
     loss_fct = CrossEntropyLoss()
     lm_loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), labels.view(-1))
 ```

 ### Упакованный набор данных

 ****

 **ВАЖНОЕ ОБНОВЛЕНИЕ**: `ConstantLengthDataset` был удален, начиная с версии `trl` 0.20. Упаковка уже токенизированного набора данных теперь выполняется функцией `pack_dataset()` из `trl.data_utils`. Возможно приблизить поведение упаковки предыдущих версий `trl`, установив `packing_strategy='wrapped'` в `SFTConfig`.

 ****



 ****

 **Итог раздела "Упакованный набор данных"**

 *   Упаковка **конкатенирует** последовательности и **разделяет** их на **равные по размеру блоки**:

     *   **Не используются токены дополнения**.

     *   Длина каждого блока не должна превышать **максимальную длину последовательности модели**.

 *   Упаковка изначально поддерживается `SFTTrainer`:

     *   Установите его аргумент `packing` в `True`.

     *   ~Он создает внутренний `ConstantLengthDataset` для обработки упаковки~ [удалено в v0.20].

     *   Он использует функцию `pack_dataset()` для обработки упаковки.

     *   Вы можете установить `packing_strategy` в `wrapped`, чтобы приблизить исходное поведение упаковки.

     *   По умолчанию **нельзя одновременно использовать упаковку и сборщик**.

 *   Некоторые **сборщики могут эффективно упаковывать** последовательности:

     *   В этом случае **аргумент `packing` должен быть установлен в `False`**, а сборщик выполняет упаковку.

     *   `DataCollatorWithFlattening` — это эквивалент упаковки для `DataCollatorForLanguageModeling`.

     *   `DataCollatorForCompletionOnlyLM` включает новый аргумент (`padding_free`), который заставляет сборщик, ориентированный только на завершение, работать подобно упаковке.

     *   Некоторые модели (например, Llama, Phi, Mistral, Gemma, OLMo и некоторые другие) поддерживают эти сборщики с Flash Attention 2:

       *   Эти модели используют `position_ids` для **обозначения границ** между исходными последовательностями, упакованными вместе.

 ****



 ![](https://github.com/dvgodoy/FineTuningLLMs/blob/main/images/ch4/packed_seq.png?raw=True)



 <center>Рисунок 4.7 - Упакованные последовательности</center>

In [48]:
sequences = dataset['text']
print(sequences[:2])


['<|user|>\nThe birch canoe slid on the smooth planks.<|end|>\n<|assistant|>\nOn the smooth planks, the birch canoe slid. Yes, hrrrm.<|end|>\n<|endoftext|>', '<|user|>\nGlue the sheet to the dark blue background.<|end|>\n<|assistant|>\nGlue the sheet to the dark blue background, you must.<|end|>\n<|endoftext|>']


 ****

 **ВАЖНОЕ ОБНОВЛЕНИЕ**: `ConstantLengthDataset` был удален в `trl` v0.20, но мы воспроизводим исходное поведение ниже (класс был импортирован из `compatibility_functions.py`), чтобы сравнить его вывод с выводом новой версии, использующей `pack_dataset()` со стратегией `wrapped`.

 ****



 **ДО**: `ConstantLengthDataset`

In [49]:
iterator = ConstantLengthDataset(tokenizer_phi, dataset,
                                 dataset_text_field='text',
                                 seq_length=64, shuffle=False)

def data_generator(iterator):
    yield from iterator

packed_dataset = Dataset.from_generator(
    data_generator,
    gen_kwargs={"iterator": iterator}
)
packed_dataset


Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 351
})

In [50]:
input_ids = packed_dataset['input_ids']
tokenizer_phi.decode(input_ids[0])


'<|user|> The birch canoe slid on the smooth planks.<|end|><|assistant|> On the smooth planks, the birch canoe slid. Yes, hrrrm.<|end|><|endoftext|><|endoftext|><|user|> Glue the sheet to the dark blue background.<|end|><|assistant|> Glue the sheet to the dark blue background, you must'

 **ПОСЛЕ**: `pack_dataset()`

In [51]:
new_packed_dataset = pack_dataset(tokenized_dataset, seq_length=64, strategy='wrapped')
new_packed_dataset


Dataset({
    features: ['input_ids'],
    num_rows: 341
})

In [52]:
input_ids = new_packed_dataset['input_ids']
tokenizer_phi.decode(input_ids[0])


'<|user|> The birch canoe slid on the smooth planks.<|end|><|assistant|> On the smooth planks, the birch canoe slid. Yes, hrrrm.<|end|><|endoftext|><|user|> Glue the sheet to the dark blue background.<|end|><|assistant|> Glue the sheet to the dark blue background, you must.'

 ![](https://github.com/dvgodoy/FineTuningLLMs/blob/main/images/ch4/packing_flow.png?raw=True)



 <center>Рисунок 4.8 - Выбор правильной конфигурации для ваших данных</center>

 #### Сборщики для упаковки

 ##### `DataCollatorWithFlattening`



 ![](https://github.com/dvgodoy/FineTuningLLMs/blob/main/images/ch4/collator_flat.png?raw=True)



 <center>Рисунок 4.9 - Сборщик, подобный упаковке</center>

In [53]:
flat_collator = DataCollatorWithFlattening()
flat_dloader = DataLoader(tokenized_dataset, batch_size=2, collate_fn=flat_collator)
flat_batch = next(iter(flat_dloader))
flat_batch


{'input_ids': tensor([[32010,   450, 29773,   305,   508,  7297,  2243,   333,   373,   278,
          10597,   715,  1331, 29889, 32007, 32001,  1551,   278, 10597,   715,
           1331, 29892,   278, 29773,   305,   508,  7297,  2243,   333, 29889,
           3869, 29892,   298, 21478,  1758, 29889, 32007, 32000, 32010,  8467,
            434,   278,  9869,   304,   278,  6501,  7254,  3239, 29889, 32007,
          32001,  8467,   434,   278,  9869,   304,   278,  6501,  7254,  3239,
          29892,   366,  1818, 29889, 32007, 32000]]),
 'labels': tensor([[ -100,   450, 29773,   305,   508,  7297,  2243,   333,   373,   278,
          10597,   715,  1331, 29889, 32007, 32001,  1551,   278, 10597,   715,
           1331, 29892,   278, 29773,   305,   508,  7297,  2243,   333, 29889,
           3869, 29892,   298, 21478,  1758, 29889, 32007, 32000,  -100,  8467,
            434,   278,  9869,   304,   278,  6501,  7254,  3239, 29889, 32007,
          32001,  8467,   434,   278,  986

In [54]:
flat_batch['input_ids'].shape, flat_batch['position_ids'].max() + 1


(torch.Size([1, 66]), tensor(38))

 ##### `DataCollatorForCompletionOnlyLM`

In [55]:
response_template = '<|assistant|>'
completion_nopad_collator = DataCollatorForCompletionOnlyLM(response_template=response_template,
                                                            tokenizer=tokenizer_phi,
                                                            padding_free=True)
completion_nopad_dloader = DataLoader(tokenized_dataset, batch_size=2, collate_fn=completion_nopad_collator)
completion_nopad_batch = next(iter(completion_nopad_dloader))
completion_nopad_batch


{'input_ids': tensor([[32010,   450, 29773,   305,   508,  7297,  2243,   333,   373,   278,
         10597,   715,  1331, 29889, 32007, 32001,  1551,   278, 10597,   715,
          1331, 29892,   278, 29773,   305,   508,  7297,  2243,   333, 29889,
          3869, 29892,   298, 21478,  1758, 29889, 32007, 32000, 32010,  8467,
           434,   278,  9869,   304,   278,  6501,  7254,  3239, 29889, 32007,
         32001,  8467,   434,   278,  9869,   304,   278,  6501,  7254,  3239,
         29892,   366,  1818, 29889, 32007, 32000]]), 'labels': tensor([[ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  1551,   278, 10597,   715,
          1331, 29892,   278, 29773,   305,   508,  7297,  2243,   333, 29889,
          3869, 29892,   298, 21478,  1758, 29889, 32007, 32000,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  8467,   434,   278,  9869,   304,   

 ### Продвинутый уровень: BYOT (Принеси свой собственный шаблон)

 ****

 **Итог раздела "Продвинутый уровень: BYOT"**

 *   Каждый шаблон должен определять **шаблон ответа** и, в идеале, **заканчиваться токеном `EOS`**.

 *   Дважды проверьте токены `EOS`, `PAD` и `UNK` вашего токенизатора:

     *   Токен `EOS` должен отличаться от токенов `PAD` и `UNK`.

     *   Токены `PAD` и `UNK` могут быть одинаковыми.

 *   **Изменяйте размер слоя эмбеддингов** только в случае крайней необходимости — т.е. если все «пустые слоты» уже использованы:

     *   При вызове `resize_token_embeddings()` модели используйте аргумент `pad_to_multiple_of`, чтобы обеспечить, чтобы размер оставался **кратным степени двойки**.

 *   Если вы не хотите создавать шаблон Jinja самостоятельно, вы можете использовать стандартный шаблон, например ChatML.

     *   Пакет `trl` предоставляет функцию `setup_chat_format()`, но у нее есть недостатки:

       *   Она назначает токен `EOS` в качестве токена `PAD` (вам нужно будет **исправить это вручную** после).

       *   Она по умолчанию изменяет размер слоя эмбеддингов модели, даже если только для его уменьшения (хотя **вы можете избежать изменения размера**, выбрав подходящий `resize_to_multiple_of`).

 *   Вы можете определить и применить **пользовательский шаблон с помощью функции форматирования** вместо создания шаблона Jinja для вашего токенизатора:

     *   Если вы укажете `formatting_func` в классе `SFTTrainer` (см. Главу 5), вашему токенизатору не нужен шаблон чата.

     *   Тщательно выбирайте шаблон ответа:

       *   Использование **обычных слов** (например, "## Ответ:") **может вызвать проблемы**, поскольку некоторые токенизаторы являются «контекстно-зависимыми» и могут разделить ваш шаблон ответа на несколько токенов.

       *   Создание **дополнительного специального токена для вашего шаблона ответа** безопаснее, так как он будет закодирован как **один токен**.

 ****

 #### Шаблон чата

In [56]:
model_opt = AutoModelForCausalLM.from_pretrained(model_dir)
tokenizer_opt = AutoTokenizer.from_pretrained(model_dir)

print(tokenizer_opt.chat_template)


None


In [57]:
tokenizer_opt.special_tokens_map


{'bos_token': '</s>',
 'eos_token': '</s>',
 'unk_token': '</s>',
 'pad_token': '<pad>'}

 **ChatML**

 ****

 [ChatML](https://github.com/openai/openai-python/blob/release-v0.28.0/chatml.md), сокращение от Chat Markup Language, был разработан OpenAI:



 _____

 "*Традиционно модели GPT потребляли неструктурированный текст. Вместо этого модели ChatGPT ожидают структурированный формат, называемый Chat Markup Language (сокращенно ChatML). Документы ChatML состоят из последовательности сообщений.*"

 _____



 Каждое сообщение должно содержать роль участника и соответствующее содержимое, как в диалоговом формате, представленном ранее. Это шаблон Jinja для ChatML:



 ```
 {% for message in messages %}
   {{'<|im_start|>' + message['role'] + '\n' + message['content'] + '<|im_end|>' + '\n'}}
 {% endfor %}
 ```

 ****

In [58]:
len(tokenizer_opt)


50265

In [59]:
model_opt.config.vocab_size


50272

In [60]:
def get_multiple_of(vocab_size):
    return 2**(bin(vocab_size)[::-1].find('1'))

pad_to_multiple_of = get_multiple_of(model_opt.config.vocab_size)
pad_to_multiple_of


32

In [61]:
model_opt.resize_token_embeddings(len(tokenizer_opt),
                                  pad_to_multiple_of=pad_to_multiple_of)


Embedding(50272, 512, padding_idx=1)

In [62]:
def modify_tokenizer(tokenizer,
                     alternative_bos_token='<|im_start|>',
                     alternative_unk_token='<unk>',
                     special_tokens=None,
                     tokens=None):
    eos_token, bos_token = tokenizer.eos_token, tokenizer.bos_token
    pad_token, unk_token = tokenizer.pad_token, tokenizer.unk_token

    # Токен BOS должен отличаться от токена EOS
    if bos_token == eos_token:
        bos_token = alternative_bos_token

    # Токен UNK должен отличаться от токена EOS
    if unk_token == eos_token:
        unk_token = alternative_unk_token

    # Токен PAD должен отличаться от токена EOS
    # но может быть таким же, как токен UNK
    if pad_token == eos_token:
        pad_token = unk_token

    assert bos_token != eos_token, "Пожалуйста, выберите другой токен BOS."
    assert unk_token != eos_token, "Пожалуйста, выберите другой токен UNK."

    # Создает словарь для токенов BOS, PAD и UNK
    # Оставляет токен EOS таким, каким он был изначально определен
    special_tokens_dict = {'bos_token': bos_token,
                           'pad_token': pad_token,
                           'unk_token': unk_token}

    # Если есть дополнительные специальные токены, добавляем их
    if special_tokens is not None:
        if isinstance(special_tokens, list):
            special_tokens_dict.update({'additional_special_tokens': special_tokens})

    tokenizer.add_special_tokens(special_tokens_dict)

    # Если есть новые обычные (не специальные) токены для добавления
    if tokens is not None:
        if isinstance(tokens, list):
            tokenizer.add_tokens(tokens)

    return tokenizer


In [63]:
def jinja_template(tokenizer):
    return ("{% for message in messages %}"
            f"{{{{'{tokenizer.bos_token}' + message['role'] + '\n' + message['content'] + '{tokenizer.eos_token}' + '\n'}}}}"
            "{% endfor %}"
            "{% if add_generation_prompt %}"
            f"{{{{ '{tokenizer.bos_token}assistant\n' }}}}"
            "{% endif %}")

def add_template(tokenizer, chat_template=None):
    # Если шаблон чата не предоставлен, создает шаблон ChatML
    # используя токены BOS и EOS
    if chat_template is None:
        chat_template = jinja_template(tokenizer)

    # Назначает шаблон чата токенизатору
    tokenizer.chat_template = chat_template

    return tokenizer


In [64]:
def get_multiple_of(vocab_size):
    return 2**(bin(vocab_size)[::-1].find('1'))

def modify_model(model, tokenizer):
    # Если новая длина токенизатора превышает размер словаря
    # изменяет его размер, сохраняя кратность тому же значению
    if len(tokenizer) > model.config.vocab_size:
        pad_to_multiple_of = get_multiple_of(model.vocab_size)
        model.resize_token_embeddings(len(tokenizer),
                                      pad_to_multiple_of=pad_to_multiple_of)

    # Обновляет идентификаторы токенов в конфигурациях модели
    if getattr(model, "config", None) is not None:
        model.config.pad_token_id = tokenizer.pad_token_id
        model.config.bos_token_id = tokenizer.bos_token_id
        model.config.eos_token_id = tokenizer.eos_token_id
    if getattr(model, "generation_config", None) is not None:
        model.generation_config.bos_token_id = tokenizer.bos_token_id
        model.generation_config.eos_token_id = tokenizer.eos_token_id
        model.generation_config.pad_token_id = tokenizer.pad_token_id

    return model


In [65]:
tokenizer_opt = modify_tokenizer(tokenizer_opt)
tokenizer_opt = add_template(tokenizer_opt)
model_opt = modify_model(model_opt, tokenizer_opt)


In [66]:
tokenizer_opt.special_tokens_map


{'bos_token': '<|im_start|>',
 'eos_token': '</s>',
 'unk_token': '<unk>',
 'pad_token': '<pad>'}

In [67]:
len(tokenizer_opt)


50266

In [68]:
tokenizer_opt.convert_ids_to_tokens(50265)


'<|im_start|>'

In [69]:
model_opt.get_input_embeddings()


Embedding(50272, 512, padding_idx=1)

In [70]:
print(tokenizer_opt.chat_template)


{% for message in messages %}{{'<|im_start|>' + message['role'] + '
' + message['content'] + '</s>' + '
'}}{% endfor %}{% if add_generation_prompt %}{{ '<|im_start|>assistant
' }}{% endif %}


In [71]:
messages = ds_msg['messages'][0]
print(tokenizer_opt.apply_chat_template(messages, tokenize=False))


<|im_start|>user
What is the capital of Argentina?</s>
<|im_start|>assistant
Buenos Aires.</s>



 #### Пользовательский шаблон

In [72]:
model_opt = AutoModelForCausalLM.from_pretrained(model_dir)
tokenizer_opt = AutoTokenizer.from_pretrained(model_dir)

response_template = '##[YODA]##>'
tokenizer_opt = modify_tokenizer(tokenizer_opt, special_tokens=[response_template])
model_opt = modify_model(model_opt, tokenizer_opt)


In [73]:
def formatting_func_builder(response_template):
    def formatting_func(examples, add_generation_prompt=False):
        output_texts = []
        for i in range(len(examples['prompt'])):
            text = f"{examples['prompt'][i]}"
            try:
                text += f" {response_template} {examples['completion'][i]}{tokenizer_opt.eos_token}"
            except KeyError:
                if add_generation_prompt:
                    text += f" {response_template} "
            output_texts.append(text)
        return output_texts
    return formatting_func

yoda_formatting_func = formatting_func_builder(response_template)
yoda_formatting_func


<function __main__.formatting_func_builder.<locals>.formatting_func(examples, add_generation_prompt=False)>

In [74]:
dataset = load_dataset("dvgodoy/yoda_sentences", split="train")
dataset = dataset.rename_column("sentence", "prompt")
dataset = dataset.rename_column("translation_extra", "completion")

formatted_seqs = yoda_formatting_func(dataset)
formatted_seqs[0]


'The birch canoe slid on the smooth planks. ##[YODA]##> On the smooth planks, the birch canoe slid. Yes, hrrrm.</s>'

In [75]:
tokenizer_opt(formatted_seqs[0])


{'input_ids': [2, 133, 23629, 611, 31728, 13763, 15, 5, 6921, 563, 2258, 4, 1437, 50266, 374, 5, 6921, 563, 2258, 6, 5, 23629, 611, 31728, 13763, 4, 3216, 6, 1368, 28015, 22900, 4, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [76]:
tokenizer_opt.convert_ids_to_tokens(50266)


'##[YODA]##>'

In [77]:
yoda_formatting_func({'prompt': ['The Force is strong in you.',
                                 'I am your father!']},
                     add_generation_prompt=True)


['The Force is strong in you. ##[YODA]##> ', 'I am your father! ##[YODA]##> ']

 #### Преимущества специальных токенов

In [ ]:
tokenizer_llama = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")
tokenizer_llama.pad_token = tokenizer_llama.unk_token
tokenizer_llama.pad_token_id = tokenizer_llama.unk_token_id


In [ ]:
prompt = """### User: Hello\n\n### Assistant: Hi, how can I help you?"""
print(prompt)


In [ ]:
tokens = tokenizer_llama.tokenize(prompt, add_special_tokens=False)
token_ids = tokenizer_llama.encode(prompt, add_special_tokens=False)
list(zip(tokens, token_ids))[6:11]


In [ ]:
response_template = "### Assistant:"
tokens = tokenizer_llama.tokenize(response_template, add_special_tokens=False)
token_ids = tokenizer_llama.encode(response_template, add_special_tokens=False)
list(zip(tokens, token_ids))


In [ ]:
dummy_ds = Dataset.from_dict({'text': [prompt]})
dummy_tokenized = dummy_ds.map(lambda row: tokenizer_llama(row['text'])).select_columns(['input_ids'])

response_template = "### Assistant:"

bad_collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer_llama)
bad_dloader = DataLoader(dummy_tokenized, batch_size=1, collate_fn=bad_collator)
bad_batch = next(iter(bad_dloader))
bad_batch


In [ ]:
modified_response_template = "\n### Assistant:"
tokens = tokenizer_llama.tokenize(modified_response_template, add_special_tokens=False)
token_ids = tokenizer_llama.encode(modified_response_template, add_special_tokens=False)
list(zip(tokens, token_ids))


In [ ]:
fixed_token_ids = token_ids[2:]
fixed_collator = DataCollatorForCompletionOnlyLM(fixed_token_ids, tokenizer=tokenizer_llama)
fixed_dloader = DataLoader(dummy_tokenized, batch_size=1, collate_fn=fixed_collator)
fixed_batch = next(iter(fixed_dloader))
fixed_batch


In [ ]:
response_template = "### Assistant:"
tokenizer_llama.add_special_tokens({'additional_special_tokens': [response_template]})


In [ ]:
dummy_tokenized = dummy_ds.map(lambda row: tokenizer_llama(row['text'])).select_columns(['input_ids'])


In [ ]:
special_collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer_llama)
special_dloader = DataLoader(dummy_tokenized, batch_size=1, collate_fn=special_collator)
special_batch = next(iter(special_dloader))
special_batch


 ### Что ждет в книге "Fine-Tuning LLMs"

 Шаблоны чата — ключ к обузданию необузданных LLM-монстров и обучению их правильному общению с нами, людьми. Умелое размещение подсказок, или специальных токенов, в диалоге позволяет им научиться отвечать, когда их активирует правильное ключевое слово. Однако процедура обучения не лишена опасностей: активации, градиенты и оптимизатор — все требуют огромных объемов драгоценной оперативной памяти для выполнения своей работы. Умиротворение этих жаждущих памяти компонентов потребует как мастерства, так и усилий. Настройка цикла обучения — не для слабонервных. Не пропустите следующую сложную главу "Fine-Tuning LLMs".